# <center>**Import necessary libraries**</center>


In [117]:
import pathlib
import pandas as pd
import design_functions.rebar_information as rebar_func
import importlib
import numpy as np
importlib.reload(rebar_func)

<module 'design_functions.rebar_information' from 'c:\\Users\\adnan.a\\Documents\\GitHub\\Killa-Design-Structures-code\\design_functions\\rebar_information.py'>

## <center>Import and read the desired beam flexural and shear reinforcement excel file from ETABS</center>

In [118]:
excel_file = pathlib.Path('excel_files/test_run.xlsx')
initial_df = pd.read_excel(excel_file, sheet_name=None)
pd.set_option('display.max_rows', None)

## <center>Because the excel sheet has two sheets, flexural and shear, they need to be read individually</center>

## <center>Delete unnecessary rows and columns for simplicity</center>

V1 of respective df's = first two rows are deleted<br>
V2 of respective df's = unique name, negative moment, negative combo, positive moment, and positive combo columns are removed.<br>
V3 of respective df's = each unique beam is consolidated to extract information such as section size, unqiue name.<br>
V4 of respective df's = ETABs section column is removed.<br>
V5 of respective df's = V2 df but all columns are removed except bottom area reinforcement and top area reinforcement<br>
V6 of respective df's = calculation DF including 100 indices and all rebar requirements<br>
For shear, identifying columns are removed as they are not needed. Additionally, unique beams DF doesn't need to be made as they can be taken from flexural DF. Therefore V2 may be primarily used

In [119]:
v1_flexural_df = initial_df['Conc Bm Flx Env - ACI 318-19'].drop([0,1])
v1_shear_df = initial_df['Conc Bm Shr Env - ACI 318-19'].drop([0,1])
v1_flexural_df.reset_index(drop=True, inplace=True)
v1_shear_df.reset_index(drop=True, inplace=True)

In [120]:
v2_flexural_df = v1_flexural_df.drop(['Unnamed: 2', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8', 'Unnamed: 9'], axis=1).copy()
v2_shear_df = v1_shear_df.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 11', 'Unnamed: 12'], axis=1).copy()

In [121]:
v3_flexural_df = v2_flexural_df.iloc[::3].copy()

## <center>Create width and depth columns to utilise in calculations</center>

In [122]:
v2_flexural_df.reset_index(drop=True, inplace=True)

In [123]:
v3_flexural_df.insert(3, 'Width (mm)', None)
v3_flexural_df.insert(4, 'Depth (mm)', None)

In [124]:
v3_flexural_df.loc[:, 'Width (mm)'] = v3_flexural_df['Unnamed: 3'].apply(rebar_func.clean_width_dimensions)

In [125]:
v3_flexural_df.loc[:, 'Depth (mm)'] = v3_flexural_df['Unnamed: 3'].apply(rebar_func.clean_depth_dimensions)

## <center>With the width and depth defined, remove the ETABS section, location, and rebar area columns</center>

In [126]:
v4_flexural_df = v3_flexural_df.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 7', 'Unnamed: 10'], axis=1).copy()

In [127]:
v4_flexural_df = v4_flexural_df.rename(columns={'Unnamed: 1': 'ETABS beam ID'})
v2_shear_df = v2_shear_df.rename(columns={'Unnamed: 7': 'Shear rebar area (mm2/m)', 'Unnamed: 10': 'Shear tension rebar area (mm2/m)', 'Unnamed: 13': 'Longitudinal tension rebar area (mm2)'})

## <center>Create Dataframe to store beam reinforcement schedule</center>

In [128]:
# Create dataframe which reflects column headings
columns = pd.MultiIndex.from_tuples([
    ('Storey', ''),
    ('Etabs ID', ''),
    ('Dimensions', 'Width (mm)'),
    ('Dimensions', 'Depth (mm)'),
    ('Bottom Reinforcement', 'Left (BL)'),
    ('Bottom Reinforcement', 'Middle (B)'),
    ('Bottom Reinforcement', 'Right (BR)'),
    ('Top Reinforcement', 'Left (TL)'),
    ('Top Reinforcement', 'Middle (T)'),
    ('Top Reinforcement', 'Right (TR)'),
    ('Side Face Reinforcement', 'Left'),
    ('Side Face Reinforcement', 'Middle'),
    ('Side Face Reinforcement', 'Right'),
    ('Shear links', 'Left (H)'),
    ('Shear links', 'Middle (J)'),
    ('Shear links', 'Right (K)')
])
beam_schedule_df = pd.DataFrame(columns=columns)

## <center>For each index of the v4 respective df, populate the main beam_schedule_df with the necessary information</center>

In [129]:
beam_schedule_df['Storey'] = v4_flexural_df['TABLE:  Concrete Beam Flexure Envelope - ACI 318-19']
beam_schedule_df['Etabs ID'] = v4_flexural_df['ETABS beam ID']
beam_schedule_df['Dimensions', 'Width (mm)'] = v4_flexural_df['Width (mm)']
beam_schedule_df['Dimensions', 'Depth (mm)'] = v4_flexural_df['Depth (mm)']
beam_schedule_df.reset_index(drop=True, inplace=True)

# <center>Create v5 df for flexure, including rebar areas and torsional longitudinal rebar<center>

In [130]:
v2_shear_df.reset_index(drop=True, inplace=True)
v5_flexural_df = v2_flexural_df.drop(['TABLE:  Concrete Beam Flexure Envelope - ACI 318-19', 'Unnamed: 1', 'Unnamed: 3'], axis=1).copy()
v5_flexural_df = v5_flexural_df.rename(columns={'Unnamed: 4': 'Location', 'Unnamed: 10': 'Bottom reinforcement', 'Unnamed: 7': 'Top Reinforcement'})
v5_flexural_df.reset_index(drop=True, inplace=True)
v5_flexural_df['Longitudinal tension rebar area (mm2)'] = v2_shear_df['Longitudinal tension rebar area (mm2)']

# <center> Create v6 df for flexure, including width and depth. This will be the df utilised for calculations.<center>

In [131]:
#create new columns
v5_flexural_df['Width (mm)'] = [1]*len(v5_flexural_df)
v5_flexural_df['Depth (mm)'] = [1]*len(v5_flexural_df)
v5_flexural_df['ETABS beam ID'] = [1]*len(v5_flexural_df)
#add new columns to beginning of dataframe, create v6
cols = ['ETABS beam ID', 'Width (mm)', 'Depth (mm)'] + [c for c in v5_flexural_df.columns if c not in ['ETABS beam ID', 'Width (mm)', 'Depth (mm)']]
v6_flexural_df = v5_flexural_df[cols].copy()

In [132]:
v6_flexural_df.loc[:, 'Width (mm)'] = v2_flexural_df['Unnamed: 3'].apply(rebar_func.clean_width_dimensions)
v6_flexural_df.loc[:, 'Depth (mm)'] = v2_flexural_df['Unnamed: 3'].apply(rebar_func.clean_depth_dimensions)
v6_flexural_df.insert(4, 'Flexural overstressed (Combo 1)', '-')
v6_flexural_df.insert(5, 'Flexural overstressed (Combo 2)', '-')
v6_flexural_df.insert(6,'Shear overstressed', '-')
v6_flexural_df.loc[:, 'Shear overstressed'] = v1_shear_df.apply(rebar_func.shear_overstressed_check, axis=1, args=['Unnamed: 6'])
v6_flexural_df.loc[:, 'Flexural overstressed (Combo 1)'] = v1_flexural_df.apply(rebar_func.flex_overstressed_check, axis=1, args=('Unnamed: 6', 'Unnamed: 9')) #check if beam is overstressed in flexure
v6_flexural_df.loc[:, 'Flexural overstressed (Combo 2)'] = v1_flexural_df.apply(rebar_func.flex_overstressed_check, axis=1, args=('Unnamed: 6', 'Unnamed: 9')) #check if beam is overstressed in flexure
v6_flexural_df.loc[:, 'ETABS beam ID'] = v2_flexural_df['Unnamed: 1']
v6_flexural_df.insert(8, 'Residual Rebar (mm2)', '-')
rebar_func.side_face_assessment(v6_flexural_df, 'Depth (mm)', 'Longitudinal tension rebar area (mm2)', 'Bottom reinforcement', 'Top Reinforcement')
v6_flexural_df.loc[:, 'Rebar Count'] = v6_flexural_df.loc[:, 'Width (mm)'].apply(rebar_func.rebar_count)
v6_flexural_df.loc[:, 'Bottom Rebar Schedule'] = v6_flexural_df.apply(rebar_func.rebar_string, axis=1, args=('Rebar Count', 'Bottom reinforcement', 'Flexural overstressed (Combo 1)', 'Flexural overstressed (Combo 2)'))
v6_flexural_df.loc[:, 'Top Rebar Schedule'] = v6_flexural_df.apply(rebar_func.rebar_string, axis=1, args=('Rebar Count', 'Top Reinforcement', 'Flexural overstressed (Combo 1)', 'Flexural overstressed (Combo 2)'))
v6_flexural_df.insert(9,'Bottom Rebar Area Provided (mm2)', v6_flexural_df.apply(rebar_func.rebar_area, axis=1, args=('Rebar Count', 'Bottom reinforcement', 'Flexural overstressed (Combo 1)', 'Flexural overstressed (Combo 2)')))
v6_flexural_df.insert(11,'Top Rebar Area Provided (mm2)', v6_flexural_df.apply(rebar_func.rebar_area, axis=1, args=('Rebar Count', 'Top Reinforcement', 'Flexural overstressed (Combo 1)', 'Flexural overstressed (Combo 2)'))) 
v6_flexural_df.loc[:, 'Residual Rebar (mm2)'] = v6_flexural_df.apply(lambda row: rebar_func.residual_rebar(row, 'Bottom Rebar Area Provided (mm2)', 'Bottom reinforcement', 'Top Rebar Area Provided (mm2)', 'Top Reinforcement'), axis=1)
v6_flexural_df.loc[:, 'Side Face Clear Space (mm)'] = v6_flexural_df['Depth (mm)'].apply(rebar_func.side_face_count)
v6_flexural_df.insert(14, 'Side Face Reinforcement Provided (mm2)', '-')
v6_flexural_df.loc[:, 'Side Face Reinforcement Provided (mm2)'] = v6_flexural_df.apply(rebar_func.side_face_area, axis=1, args=('Longitudinal tension rebar area (mm2)', 'Side Face Clear Space (mm)', 'Residual Rebar (mm2)', 'Shear overstressed'))
v6_flexural_df.loc[:, 'Side Face Reinforcement Schedule'] = v6_flexural_df.apply(rebar_func.side_face_reinf, axis=1, args=('Longitudinal tension rebar area (mm2)', 'Side Face Clear Space (mm)', 'Residual Rebar (mm2)', 'Shear overstressed'))
move_1, move_2, move_3, move_4 = v6_flexural_df.pop('Residual Rebar (mm2)'), v6_flexural_df.pop('Top Rebar Area Provided (mm2)'), v6_flexural_df.pop('Bottom Rebar Area Provided (mm2)'), v6_flexural_df.pop('Side Face Clear Space (mm)')
v6_flexural_df.insert(8, 'Top Rebar Area Provided (mm2)', move_2)
v6_flexural_df.insert(10, 'Bottom Rebar Area Provided (mm2)', move_3)
v6_flexural_df.insert(11, 'Residual Rebar (mm2)', move_1)
v6_flexural_df.insert(14, 'Side Face Clear Space (mm)', move_4)

## <center>Create calculation process for shear reinforcement<center>

In [133]:
v2_shear_df.loc[:, 'Shear rebar area (mm2/m)'] = v2_shear_df['Shear rebar area (mm2/m)'].fillna('O/S')
v2_shear_df.loc[:, 'Required Shear Area (mm2)'] = v2_shear_df.apply(rebar_func.shear_area_req, axis=1, args=('Shear rebar area (mm2/m)', 'Shear tension rebar area (mm2/m)'))
v2_shear_df.insert(1, 'Width (mm)', v6_flexural_df['Width (mm)'])
v2_shear_df.insert(6, 'Required Shear Legs', '-')
v2_shear_df.loc[:, 'Required Shear Legs'] = v2_shear_df['Width (mm)'].apply(rebar_func.req_legs)
v2_shear_df.insert(7, 'Shear Link Schedule', '-')
v2_shear_df.loc[:, 'Shear Link Schedule'] = v2_shear_df.apply(rebar_func.shear_string, axis=1, args=('Required Shear Legs', 'Required Shear Area (mm2)', 'Shear rebar area (mm2/m)'))

# <center>With all the flexural and shear scheduling gathered, transpose and input the information into the beam schedule DF<center>

In [134]:
#from v6 create v7
#only includes bottom, top, and side face sched
v7_flexural_df = v6_flexural_df.drop(['Width (mm)', 'Depth (mm)', 'Location', 'Bottom reinforcement', 'Top Reinforcement', 'Longitudinal tension rebar area (mm2)', 'Rebar Count', 'Side Face Clear Space (mm)', 'Residual Rebar (mm2)', 'Bottom Rebar Area Provided (mm2)', 'Top Rebar Area Provided (mm2)'], axis=1).copy()
v7_flexural_df.reset_index(drop=True, inplace=True)
v3_shear_df = v2_shear_df.drop(['Width (mm)', 'Shear rebar area (mm2/m)', 'Shear tension rebar area (mm2/m)', 'Longitudinal tension rebar area (mm2)', 'Required Shear Area (mm2)', 'Required Shear Legs'], axis=1).copy()
v3_shear_df = v3_shear_df.iloc[:, 1:].copy()
v3_shear_df.reset_index(drop=True, inplace=True)

side_face_df = v7_flexural_df.drop(['ETABS beam ID', 'Flexural overstressed (Combo 1)', 'Flexural overstressed (Combo 2)', 'Shear overstressed', 'Top Rebar Schedule', 'Bottom Rebar Schedule'], axis=1).copy()

def replace_with_max(group):
    # Convert 'Side Face Reinforcement Provided (mm2)' to numeric, making non-numeric values NaN
    group['Side Face Reinforcement Provided (mm2)'] = pd.to_numeric(group['Side Face Reinforcement Provided (mm2)'], errors='coerce')

    # If all values are NaN, leave the group as it is
    if group['Side Face Reinforcement Provided (mm2)'].isna().all():
        return group

    # Find the row with the maximum value
    max_row = group.loc[group['Side Face Reinforcement Provided (mm2)'].idxmax()]

    # Replace all values in the group with the values from the max row
    group['Side Face Reinforcement Provided (mm2)'] = max_row['Side Face Reinforcement Provided (mm2)']
    group['Side Face Reinforcement Schedule'] = max_row['Side Face Reinforcement Schedule']
    return group

# Create a 'Group' column for grouping
side_face_df['Group'] = np.repeat(range(len(side_face_df) // 3), 3)

# Apply the function to each group
side_face_df = side_face_df.groupby('Group').apply(replace_with_max)

# Drop the 'Group' column
side_face_df = side_face_df.drop(columns='Group')

side_face_df = side_face_df.reset_index(drop=True)

#update v7_flexural_df with new side face reinf schedule

side_face_df = side_face_df.drop(columns='Side Face Reinforcement Provided (mm2)')

v7_flexural_df['Side Face Reinforcement Schedule'] = side_face_df['Side Face Reinforcement Schedule']

In [135]:
v3_shear_df['new_index'] = np.repeat(range(len(v3_shear_df) // 3), 3)[:len(v3_shear_df)]
v3_shear_df_grouped = v3_shear_df.groupby('new_index')['Shear Link Schedule'].apply(list).apply(pd.Series).reset_index(drop=True)

v7_flexural_df['new_index'] = np.repeat(range(len(v7_flexural_df) // 3), 3)[:len(v7_flexural_df)]
v7_flexural_df_grouped_Bottom_Rebar = v7_flexural_df.groupby('new_index')['Bottom Rebar Schedule'].apply(list).apply(pd.Series).reset_index(drop=True).copy()
v7_flexural_df_grouped_Top_Rebar = v7_flexural_df.groupby('new_index')['Top Rebar Schedule'].apply(list).apply(pd.Series).reset_index(drop=True).copy()
v7_flexural_df_grouped_Side_Rebar = v7_flexural_df.groupby('new_index')['Side Face Reinforcement Schedule'].apply(list).apply(pd.Series).reset_index(drop=True).copy()
v7_flexural_df_grouped_Bottom_Rebar.head()

,0,1,2
0,2 rows of 5T16,3 rows of 5T16,3 rows of 5T16
1,2 rows of 5T16,2 rows of 5T16,2 rows of 5T16
2,2T16,2T16,2T16
3,2T16,2T16,2T16
4,2T16,2T16,2T16


In [136]:
beam_schedule_df[('Bottom Reinforcement', 'Left (BL)')].update(v7_flexural_df_grouped_Bottom_Rebar[0])
beam_schedule_df[('Bottom Reinforcement', 'Middle (B)')].update(v7_flexural_df_grouped_Bottom_Rebar[1])
beam_schedule_df[('Bottom Reinforcement', 'Right (BR)')].update(v7_flexural_df_grouped_Bottom_Rebar[2])
beam_schedule_df[('Top Reinforcement', 'Left (TL)')].update(v7_flexural_df_grouped_Top_Rebar[0])
beam_schedule_df[('Top Reinforcement', 'Middle (T)')].update(v7_flexural_df_grouped_Top_Rebar[1])
beam_schedule_df[('Top Reinforcement', 'Right (TR)')].update(v7_flexural_df_grouped_Top_Rebar[2])
beam_schedule_df[('Side Face Reinforcement', 'Left')].update(v7_flexural_df_grouped_Side_Rebar[0])
beam_schedule_df[('Side Face Reinforcement', 'Middle')].update(v7_flexural_df_grouped_Side_Rebar[1])
beam_schedule_df[('Side Face Reinforcement', 'Right')].update(v7_flexural_df_grouped_Side_Rebar[2])
beam_schedule_df[('Shear links', 'Left (H)')].update(v3_shear_df_grouped[0])
beam_schedule_df[('Shear links', 'Middle (J)')].update(v3_shear_df_grouped[1])
beam_schedule_df[('Shear links', 'Right (K)')].update(v3_shear_df_grouped[2])
# beam_schedule_df
beam_schedule_df.to_excel('Mausam Check_v2.xlsx')